![Logo Pyrates LLM](../assets/pyratesllm_logo_500.png)

# **Notebook#02 : Analysis on axis 1**
## [Impact of the digital assistant on the learner]

## 1/ Imports

In [ ]:
# Internal
import sys
sys.path.append("../src")
import students_constants as stu_const
import interaction_constants as int_const
import tests_constants  as tes_const

# External
import pandas as pd
from scipy import stats
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt

## 2/ Data importation

In [ ]:
interaction_data = pd.read_pickle("../data/interim/interaction_data.pkl")
pre_test_data = pd.read_pickle("../data/interim/pre_test_data.pkl")
post_test_data = pd.read_pickle("../data/interim/post_test_data.pkl")

## 3/ Game progression (Q1.1) [A/B/C]

### 3.1/ Calculate game progression (index 0-100)

In [ ]:
# Get all STARTED actions to find maximum level reached by each student
started_actions = interaction_data[
    interaction_data[int_const.ACTION_DATA_KEY] == int_const.STARTED_ACTION
]

# Find maximum level reached by each student
max_level_per_student = started_actions.groupby(
    int_const.GAME_ID_DATA_KEY
)[int_const.LEVEL_DATA_KEY].max()

print(f"Number of games detected: {len(max_level_per_student)}")

student_progress_data = []

for game_id in max_level_per_student.index:
    max_level = max_level_per_student[game_id]
    
    # Get all traces for this student at his maximum level
    student_level_data = interaction_data[
        (interaction_data[int_const.GAME_ID_DATA_KEY] == game_id) & 
        (interaction_data[int_const.LEVEL_DATA_KEY] == max_level)
    ]
    
    # Maximum progression
    max_progression = student_level_data[int_const.GAME_PROGRESSION_DATA_KEY].max()
    
    # Group id
    group = student_level_data[int_const.GROUP_ID_DATA_KEY].iloc[0]

    # Results aggregation 
    student_progress_data.append({
        'game_id': game_id,
        'group_id': group,
        'max_level': max_level,
        'max_progression': max_progression
    })

# Create progress dataframe
progress_df = pd.DataFrame(student_progress_data)

# Calculate progress index (0-100 points)
# - Each completed level gives 12.5 points (100/8)
# - For current level, progression percentage gives fraction of 12.5 points
def calculate_progress_index(max_level, max_progression):
    completed_levels_points = (max_level - 1) * (100 / 8)
    current_level_points = (max_progression / 100) * (100 / 8)
    return min(completed_levels_points + current_level_points, 100)

progress_df['progress_index'] = progress_df.apply(
    lambda row: calculate_progress_index(row['max_level'], row['max_progression']),
    axis=1
)

print(f"Number of progress entries computed: {len(progress_df)}")
# Export to Excel for debug
progress_df.to_excel("../debug/debug_students_game_progression.xlsx")


### 3.2/ Test A VS (B+C)

Is there a difference in game progression between students from groups A and students from group B+C ?

In [ ]:
# Extract progress data for each group
group_a_progress = progress_df[progress_df['group_id'] == 'A']['progress_index']
group_b_progress = progress_df[progress_df['group_id'] == 'B']['progress_index']
group_c_progress = progress_df[progress_df['group_id'] == 'C']['progress_index']
# Combine B and C into one group
group_bc_progress = pd.concat([group_b_progress, group_c_progress])

# Descriptive statistics
average_progress_A = group_a_progress.mean()
average_progress_BC = group_bc_progress.mean()

std_progress_A = group_a_progress.std()
std_progress_BC = group_bc_progress.std()

mean_diff = average_progress_BC - average_progress_A
percent_increase = (mean_diff / average_progress_A) * 100

print("=== DESCRIPTIVE STATISTICS ===")
print(f"Group A: n = {len(group_a_progress)}, mean = {average_progress_A:.2f}, std = {std_progress_A:.2f}")
print(f"Group B+C: n = {len(group_bc_progress)}, mean = {average_progress_BC:.2f}, std = {std_progress_BC:.2f}")
print(f".   Difference in mean (B+C - A): {mean_diff:.2f}")
print(f".   Percentage increase from Group A: {percent_increase:.2f}%")

alpha=0.05
print(f"\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(group_a_progress)
shapiro_BC = stats.shapiro(group_bc_progress)
print(f"Shapiro A: p-value = {shapiro_A.pvalue:.4f}")
print(f"Shapiro B+C: p-value = {shapiro_BC.pvalue:.4f}")

# Normality conclusions
if shapiro_A.pvalue >= alpha:
    print("Group A: data appears normally distributed")
else:
    print("Group A: data does not appear normally distributed")

if shapiro_BC.pvalue >= alpha:
    print("Group B+C: data appears normally distributed")
else:
    print("Group B+C: data does not appear normally distributed")

print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_p = stats.levene(group_a_progress, group_bc_progress).pvalue
print(f"Levene A vs B/C p-value = {levene_p:.4f}")

# Variance conclusion
if levene_p >= alpha:
    print("Variances are similar between groups")
else:
    print("Variances differ between groups")

print(f"\n=== TEST RESULTS ===")
# Decide which test to use
normal_A = shapiro_A.pvalue >= alpha
normal_BC = shapiro_BC.pvalue >= alpha
levene_AB_C = levene_p >= 0.05

if normal_A and normal_BC and levene_AB_C:
    print("Using independent t-test (parametric)")
    t_stat, p_value = stats.ttest_ind(group_a_progress, group_bc_progress, equal_var=True)
else:
    print("Using Mann-Whitney U test (non-parametric)")
    t_stat, p_value = stats.mannwhitneyu(group_a_progress, group_bc_progress, alternative='two-sided')

print(f"Test statistic = {t_stat:.4f}")
print(f"P-value = {p_value:.4f}")

if p_value < alpha:
    print(f"SIGNIFICANT: p-value = {p_value:.4f} < alpha = {alpha}")
    print("There are statistically significant differences between Group A and Group B+C")
else:
    print(f"NOT SIGNIFICANT: p-value = {p_value:.4f} > alpha = {alpha}")
    print("No statistically significant differences between Group A and Group B+C")

### 3.3/ Test A VS B VS C

Is there a difference in game progression between students from groups A, B, and C?

In [ ]:
# Descriptive statistics
average_progress_A = group_a_progress.mean()
average_progress_B = group_b_progress.mean()
average_progress_C = group_c_progress.mean()

diff_B = average_progress_B - average_progress_A
percent_increase_B = (diff_B / average_progress_A) * 100

diff_C = average_progress_C - average_progress_A
percent_increase_C = (diff_C / average_progress_A) * 100


print("=== DESCRIPTIVE STATISTICS ===")
print(f"Group A: n = {len(group_a_progress)}, mean = {average_progress_A:.2f}, std = {group_a_progress.std():.2f}")
print(f"Group B: n = {len(group_b_progress)}, mean = {average_progress_B:.2f}, std = {group_b_progress.std():.2f}")
print(f".   diff vs A = {diff_B:.2f}, increase = {percent_increase_B:.2f}%")
print(f"Group C: n = {len(group_c_progress)}, mean = {average_progress_C:.2f}, std = {group_c_progress.std():.2f}")
print(f".   diff vs A = {diff_C:.2f}, increase = {percent_increase_C:.2f}%")

alpha = 0.05
print(f"=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(group_a_progress)
shapiro_B = stats.shapiro(group_b_progress)
shapiro_C = stats.shapiro(group_c_progress)

print(f"Group A: p-value = {shapiro_A.pvalue:.4f}")
print(f"Group B: p-value = {shapiro_B.pvalue:.4f}")
print(f"Group C: p-value = {shapiro_C.pvalue:.4f}")

# Check if all groups are normally distributed
all_normal = all(p >= alpha for p in [shapiro_A.pvalue, shapiro_B.pvalue, shapiro_C.pvalue])

if all_normal:
    print("All groups follow normal distribution -> ANOVA is applicable.")
else:
    print("At least one group is not normally distributed -> Prefer Kruskal-Wallis test.")

print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_test = stats.levene(group_a_progress, group_b_progress, group_c_progress)
print(f"Levene's test for homogeneity of variances: p-value = {levene_test.pvalue:.4f}")

if levene_test.pvalue >= alpha:
    print("Variances are homogeneous")
else:
    print("Variances are not homogeneous")


# Prepare data for overall tests
all_progress = pd.concat([
    pd.DataFrame({'group': 'A', 'progress': group_a_progress}),
    pd.DataFrame({'group': 'B', 'progress': group_b_progress}),
    pd.DataFrame({'group': 'C', 'progress': group_c_progress})
])

# ANOVA (parametric)
anova_result = stats.f_oneway(group_a_progress, group_b_progress, group_c_progress)
# One-way : one factor (group)
print(f"\n=== ONE-WAY ANOVA RESULTS ===")
print(f"F-statistic: {anova_result.statistic:.4f}")
print(f"P-value: {anova_result.pvalue:.4f}")

# Kruskal-Wallis test (non-parametric)
kruskal_result = stats.kruskal(group_a_progress, group_b_progress, group_c_progress)
print(f"\n=== KRUSKAL-WALLIS TEST RESULTS ===")
print(f"H-statistic: {kruskal_result.statistic:.4f}")
print(f"P-value: {kruskal_result.pvalue:.4f}")

# Interpretation of overall test
if all_normal and levene_test.pvalue >= alpha:
    overall_pvalue = anova_result.pvalue
    test_used = "ANOVA"
else:
    overall_pvalue = kruskal_result.pvalue
    test_used = "Kruskal-Wallis"

print(f"\n=== OVERALL INTERPRETATION (using {test_used}) ===")
if overall_pvalue < alpha:
    print(f"SIGNIFICANT: p-value = {overall_pvalue:.4f} < alpha = {alpha}")
    print("There are statistically significant differences between at least two groups")
else:
    print(f"NOT SIGNIFICANT: p-value = {overall_pvalue:.4f} > alpha = {alpha}")
    print("No statistically significant differences between groups")

# POST-HOC TESTS (if significant overall difference)
if overall_pvalue < alpha:
    print(f"\n=== POST-HOC PAIRWISE COMPARISONS ===")
    
    # For parametric data: Tukey HSD
    if test_used == "ANOVA":
        print("Tukey HSD Post-hoc Test:")
        tukey = pairwise_tukeyhsd(endog=all_progress['progress'], groups=all_progress['group'], alpha=alpha)
        print(tukey)
        
        # Extract significant pairs
        print("\nSignificant pairwise differences:")
        for i in range(len(tukey._results[0])):
            if tukey._results[4][i]:  # If reject (True)
                group1, group2 = tukey._results[1][i], tukey._results[2][i]
                meandiff = tukey._results[3][i]
                pval = tukey._results[4][i]
                print(f"  {group1} vs {group2}: SIGNIFICANT (mean diff = {meandiff:.2f}, p = {pval:.4f})")
    
    # For non-parametric data: Mann-Whitney U tests with Bonferroni correction
    else:
        print("Mann-Whitney U tests with Bonferroni correction:")
        pairs = [('A', 'B'), ('A', 'C'), ('B', 'C')]
        bonferroni_alpha = alpha / len(pairs)  # Bonferroni correction
        
        for pair in pairs:
            group1, group2 = pair
            data1 = group_a_progress if group1 == 'A' else (group_b_progress if group1 == 'B' else group_c_progress)
            data2 = group_a_progress if group2 == 'A' else (group_b_progress if group2 == 'B' else group_c_progress)
            
            u_stat, p_value = stats.mannwhitneyu(data1, data2)
            adjusted_p = p_value * len(pairs)  # Bonferroni adjustment
            significance = "SIGNIFICANT" if adjusted_p < alpha else "not significant"
            
            print(f"  {group1} vs {group2}: p = {p_value:.4f}, "
                  f"adjusted p = {adjusted_p:.4f} -> {significance}")

## 4/ Learning gain (Q1.2) [A/B/C]

### 4.1/ Learning gain calculation

The learning gain calculation is based on ANSWERS_SCORES dictionary (see `scr/tests_constants.py`)

In [ ]:
# Calculate maximum score
max_score_general = sum(max(answer.values()) for answer in tes_const.ANSWERS_SCORES.values())

# Score calculation function
def calculate_score(student, max_score):
    score = 0
    for question, answer in student.items():
        if question in tes_const.ANSWERS_SCORES and answer in tes_const.ANSWERS_SCORES[question]:
            score += tes_const.ANSWERS_SCORES[question][answer]
    return round((score/max_score)*100)

groups = [tes_const.GROUP_A, tes_const.GROUP_B, tes_const.GROUP_C]
scores_data = {}

for group in groups:
    # Filter and calculate scores
    pre_temp = pre_test_data[pre_test_data[tes_const.GROUP_ID_KEY] == group].copy()
    post_temp = post_test_data[post_test_data[tes_const.GROUP_ID_KEY] == group].copy()
    
    pre_temp['pre_score'] = pre_temp.apply(calculate_score, axis=1, max_score=max_score_general)
    post_temp['post_score'] = post_temp.apply(calculate_score, axis=1, max_score=max_score_general)
    
    # Merge and calculate gain
    scores_df = pd.merge(
        pre_temp[[tes_const.STUDENT_ID_KEY, 'pre_score']],
        post_temp[[tes_const.STUDENT_ID_KEY, 'post_score']],
        on=tes_const.STUDENT_ID_KEY,
        how='inner' # the student must have a pre-test score AND a post-test score
    )
    scores_df['learning_gain'] = scores_df['post_score'] - scores_df['pre_score']
    
    scores_data[group] = scores_df
    print(f"Group {group}: {len(scores_df)}/{sum(1 for s in stu_const.ALL_STUDENTS if s[stu_const.GROUP_ID] == group)} students with both pre and post tests")
    
    # Export
    scores_df.to_excel(f"../debug/debug_learning_gain_{group}.xlsx", index=False)


### 4.2/ Intra-group learning gain analysis

Do students demonstrate learning gains using Pyrates in their respective groups?

In [ ]:
alpha = 0.05
groups = [tes_const.GROUP_A, tes_const.GROUP_B, tes_const.GROUP_C]
for group in groups:
    scores_df = scores_data[group]
    pre_scores = scores_df['pre_score']
    post_scores = scores_df['post_score']
    learning_gains = scores_df['learning_gain']
    print(f"\n=======================================")
    print(f"GROUP {group} INTRA-GROUP ANALYSIS")
    print(f"========================================")

    print("\n=== DESCRIPTIVE STATISTICS ===")

    print(f"Pre-test: M = {pre_scores.mean():.2f}, SD = {pre_scores.std():.2f}")
    print(f"Post-test: M = {post_scores.mean():.2f}, SD = {post_scores.std():.2f}")
    print(f"Learning gain: M = {learning_gains.mean():.2f}, SD = {learning_gains.std():.2f}")
    
    print(f"\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
    # Shapiro-Wilk test for pre-post differences
    shapiro_gain = stats.shapiro(learning_gains)
    print(f"Shapiro-Wilk test for learning gains: p-value = {shapiro_gain.pvalue:.4f}")
    
    # Check normality
    if shapiro_gain.pvalue >= 0.05:
        print("Learning gains are normally distributed -> Paired T-test is applicable.")
        normal_distribution = True
    else:
        print("Learning gains are not normally distributed -> Prefer Wilcoxon test.")
        normal_distribution = False
    
    print(f"\n=== TEST RESULTS ===")
    # Paired T-test
    t_test = stats.ttest_rel(pre_scores, post_scores)
    print(f"Paired T-test: t = {t_test.statistic:.4f}, p-value = {t_test.pvalue:.4f}")
    
    # Wilcoxon signed-rank test
    wilcoxon_test = stats.wilcoxon(pre_scores, post_scores)
    print(f"Wilcoxon test: W = {wilcoxon_test.statistic:.4f}, p-value = {wilcoxon_test.pvalue:.4f}")
    
    # Choose appropriate test based on normality
    if normal_distribution:
        recommended_pvalue = t_test.pvalue
        test_used = "Paired T-test"
    else:
        recommended_pvalue = wilcoxon_test.pvalue
        test_used = "Wilcoxon test"
    
    # Interpretation
    print(f"Results (using {test_used}) :")
    
    if recommended_pvalue < alpha:
        print(f"SIGNIFICANT: p-value = {recommended_pvalue:.4f} < alpha = {alpha}")
        if learning_gains.mean() > 0:
            print(f"   Significant learning gain observed (+{learning_gains.mean():.2f} points)")
        else:
            print(f"   Significant learning loss observed ({learning_gains.mean():.2f} points)")
    else:
        print(f"NOT SIGNIFICANT: p-value = {recommended_pvalue:.4f} > alpha = {alpha}")
        print("   No significant learning gain observed")


### 4.3/ Inter-group learning gain comparison

#### 4.3.1/ Test A VS (B+C)

Is there a difference in learning gains between students from groups A and those from group B+C ?

In [ ]:
# Extract learning gains
learning_gains_A = scores_data['A']['learning_gain']
learning_gains_BC = pd.concat([scores_data['B']['learning_gain'], scores_data['C']['learning_gain']])

# Descriptive statistics
mean_A = learning_gains_A.mean()
mean_BC = learning_gains_BC.mean()
std_A = learning_gains_A.std()
std_BC = learning_gains_BC.std()
diff_A_BC = mean_A - mean_BC
percent_reduction_BC = (diff_A_BC / mean_A) * 100

print("=== DESCRIPTIVE STATISTICS ===")
print(f"Group A (n={len(learning_gains_A)}): M = {mean_A:.2f}, SD = {std_A:.2f}")
print(f"Group B+C (n={len(learning_gains_BC)}): M = {mean_BC:.2f}, SD = {std_BC:.2f}")
print(f".   Difference (A - B+C) = {diff_A_BC:.2f}")
print(f".   Percentage reduction compared to A = {percent_reduction_BC:.2f}%")

alpha = 0.05
# Normality test
print("\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(learning_gains_A)
shapiro_BC = stats.shapiro(learning_gains_BC)
print(f"Group A: p-value = {shapiro_A.pvalue:.4f} ->{'normally distributed' if shapiro_A.pvalue >= alpha else 'not normally distributed'}")
print(f"Group B+C: p-value = {shapiro_BC.pvalue:.4f} -> {'normally distributed' if shapiro_BC.pvalue >= alpha else 'not normally distributed'}")

# Levene test for equal variances
print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_p = stats.levene(learning_gains_A, learning_gains_BC).pvalue
print(f"Levene p-value = {levene_p:.4f} -> {'variances are similar' if levene_p >= alpha else 'variances differ'}")


print(f"\n=== TEST RESULTS ===")
# Choose statistical test
normal_A = shapiro_A.pvalue >= alpha
normal_BC = shapiro_BC.pvalue >= alpha
equal_var = levene_p >= alpha

if normal_A and normal_BC and equal_var:
    print("Using independent t-test (parametric)")
    stat, p_value = stats.ttest_ind(learning_gains_A, learning_gains_BC, equal_var=True)
else:
    print("Using Mann-Whitney U test (non-parametric)")
    stat, p_value = stats.mannwhitneyu(learning_gains_A, learning_gains_BC, alternative='two-sided')

print(f"Test statistic = {stat:.4f}")
print(f"P-value = {p_value:.4f}")

if p_value < alpha:
    print(f"SIGNIFICANT difference: p = {p_value:.4f} < {alpha}")
else:
    print(f"No significant difference: p = {p_value:.4f} >= {alpha}")


#### 4.3.2/ Test A VS B VS C

Is there a difference in learning gains between students from groups A, B, and C?

In [ ]:
# Extract learning gains for each group
learning_gains_A = scores_data['A']['learning_gain']
learning_gains_B = scores_data['B']['learning_gain'] 
learning_gains_C = scores_data['C']['learning_gain']

# Descriptive statistics
average_gain_A = learning_gains_A.mean()
average_gain_B = learning_gains_B.mean()
average_gain_C = learning_gains_C.mean()
diff_B_vs_A = average_gain_A - average_gain_B
diff_C_vs_A = average_gain_A - average_gain_C

percent_reduction_B = (diff_B_vs_A / average_gain_A) * 100
percent_reduction_C = (diff_C_vs_A / average_gain_A) * 100

print("\n=== DESCRIPTIVE STATISTICS ===")
print(f"Group A (n={len(learning_gains_A)}): M = {average_gain_A:.2f}, SD = {learning_gains_A.std():.2f}")
print(f"Group B (n={len(learning_gains_B)}): M = {average_gain_B:.2f}, SD = {learning_gains_B.std():.2f}")
print(f".   B vs A: Mean difference = {diff_B_vs_A:.2f}, Percentage reduction = {percent_reduction_B:.2f}%")
print(f"Group C (n={len(learning_gains_C)}): M = {average_gain_C:.2f}, SD = {learning_gains_C.std():.2f}")
print(f".   C vs A: Mean difference = {diff_C_vs_A:.2f}, Percentage reduction = {percent_reduction_C:.2f}%")

alpha = 0.05
# Shapiro-Wilk normality test
print(f"\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(learning_gains_A)
shapiro_B = stats.shapiro(learning_gains_B)
shapiro_C = stats.shapiro(learning_gains_C)

print(f"Group A: p-value = {shapiro_A.pvalue:.4f}")
print(f"Group B: p-value = {shapiro_B.pvalue:.4f}")
print(f"Group C: p-value = {shapiro_C.pvalue:.4f}")

# Check if all groups are normally distributed
all_normal = all(p >= alpha for p in [shapiro_A.pvalue, shapiro_B.pvalue, shapiro_C.pvalue])

if all_normal:
    print("All groups follow normal distribution -> ANOVA is applicable.")
else:
    print("At least one group is not normally distributed -> Prefer Kruskal-Wallis test.")

print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_test = stats.levene(learning_gains_A, learning_gains_B, learning_gains_C)
print(f"Levene's test for homogeneity of variances: p-value = {levene_test.pvalue:.4f}")

if levene_test.pvalue >= alpha:
    print("Variances are homogeneous")
else:
    print("Variances are not homogeneous")

# Prepare data for overall tests
all_gains = pd.concat([
    pd.DataFrame({'group': 'A', 'gain': learning_gains_A}),
    pd.DataFrame({'group': 'B', 'gain': learning_gains_B}),
    pd.DataFrame({'group': 'C', 'gain': learning_gains_C})
])

# ANOVA (parametric)
anova_result = stats.f_oneway(learning_gains_A, learning_gains_B, learning_gains_C)
print(f"\n=== ONE-WAY ANOVA RESULTS ===")
print(f"F-statistic: {anova_result.statistic:.4f}")
print(f"P-value: {anova_result.pvalue:.4f}")

# Kruskal-Wallis test (non-parametric)
kruskal_result = stats.kruskal(learning_gains_A, learning_gains_B, learning_gains_C)
print(f"\n=== KRUSKAL-WALLIS TEST RESULTS ===")
print(f"H-statistic: {kruskal_result.statistic:.4f}")
print(f"P-value: {kruskal_result.pvalue:.4f}")

# Interpretation of overall test
if all_normal and levene_test.pvalue >= 0.05:
    overall_pvalue = anova_result.pvalue
    test_used = "ANOVA"
else:
    overall_pvalue = kruskal_result.pvalue
    test_used = "Kruskal-Wallis"

print(f"\n=== OVERALL INTERPRETATION (using {test_used}) ===")
if overall_pvalue < alpha:
    print(f"SIGNIFICANT: p-value = {overall_pvalue:.4f} < alpha = {alpha}")
    print("   There are statistically significant differences in learning gains between at least two groups")
else:
    print(f"NOT SIGNIFICANT: p-value = {overall_pvalue:.4f} > alpha = {alpha}")
    print("   No statistically significant differences in learning gains between groups")

# POST-HOC TESTS (if significant overall difference)
if overall_pvalue < alpha:
    print(f"\n=== POST-HOC PAIRWISE COMPARISONS ===")
    
    # For parametric data: Tukey HSD
    if test_used == "ANOVA":
        print("Tukey HSD Post-hoc Test:")
        tukey = pairwise_tukeyhsd(endog=all_gains['gain'], groups=all_gains['group'], alpha=alpha)
        print(tukey)
        
        # Extract significant pairs
        print("\nSignificant pairwise differences:")
        for i in range(len(tukey._results[0])):
            if tukey._results[4][i]:  # If reject (True)
                group1, group2 = tukey._results[1][i], tukey._results[2][i]
                meandiff = tukey._results[3][i]
                pval = tukey._results[4][i]
                print(f"  {group1} vs {group2}: SIGNIFICANT (mean diff = {meandiff:.2f}, p = {pval:.4f})")
    
    # For non-parametric data: Mann-Whitney U tests with Bonferroni correction
    else:
        print("Mann-Whitney U tests with Bonferroni correction:")
        pairs = [('A', 'B'), ('A', 'C'), ('B', 'C')]
        bonferroni_alpha = alpha / len(pairs)
        
        for pair in pairs:
            group1, group2 = pair
            data1 = learning_gains_A if group1 == 'A' else (learning_gains_B if group1 == 'B' else learning_gains_C)
            data2 = learning_gains_A if group2 == 'A' else (learning_gains_B if group2 == 'B' else learning_gains_C)
            
            u_stat, p_value = stats.mannwhitneyu(data1, data2)
            adjusted_p = p_value * len(pairs)
            significance = "SIGNIFICANT" if adjusted_p < alpha else "not significant"
            
            print(f"  {group1} vs {group2}: p = {p_value:.4f}, adjusted p = {adjusted_p:.4f} -> {significance}")



## 5/ Teacher solicitation (Q1.3) [A/B/C]

### 5.1/ Teacher solicitation calculation

In [ ]:
# Process all groups in one go
groups = [int_const.GROUP_A, int_const.GROUP_B, int_const.GROUP_C]
calls_data = {}

for group in groups:
    # Filter and count teacher calls
    group_data = interaction_data[interaction_data[int_const.GROUP_ID_DATA_KEY] == group]
    
    teacher_calls = group_data[
        (group_data[int_const.ACTION_DATA_KEY] == int_const.RECEIVED_ACTION) &
        (group_data[int_const.OBJECT_DATA_KEY] == int_const.TEACHER_HELP_OBJECT)
    ]
    
    # Count per student
    calls_per_student = teacher_calls.groupby(int_const.GAME_ID_DATA_KEY).size()
    
    # Get valid students and fill missing with 0
    valid_students = [s[stu_const.GAME_ID] for s in stu_const.ALL_STUDENTS if s[stu_const.GROUP_ID] == group]
    calls_per_student = calls_per_student.reindex(valid_students, fill_value=0)
    
    calls_data[group] = calls_per_student
    print(f"Group {group}: {len(calls_per_student)} students processed")
    
    # Export
    calls_per_student.to_excel(f"../debug/debug_teacher_solicitation_per_student_{group}.xlsx")

### 5.2/ Test A VS (B+C)

Is there a difference in teacher solicitation between students from groups A and students from group B+C ?

In [ ]:
# Extract calls data for each group
calls_A = calls_data['A']
calls_B = calls_data['B']
calls_C = calls_data['C']

# Combine B and C groups
calls_BC = pd.concat([calls_B, calls_C])

# Descriptive statistics
mean_A = calls_A.mean()
mean_BC = calls_BC.mean()
std_A = calls_A.std()
std_BC = calls_BC.std()
diff_A_BC = mean_A - mean_BC
percent_reduction_BC = (diff_A_BC / mean_A) * 100

print("\n=== DESCRIPTIVE STATISTICS ===")
print(f"Group A (n={len(calls_A)}): M = {mean_A:.2f}, SD = {std_A:.2f}")
print(f"Group B+C (n={len(calls_BC)}): M = {mean_BC:.2f}, SD = {std_BC:.2f}")
print(f".   Difference (A - B+C) = {diff_A_BC:.2f}")
print(f".   Percentage reduction compared to A = {percent_reduction_BC:.2f}%")

alpha = 0.05
# Shapiro-Wilk normality test
print("\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(calls_A)
shapiro_BC = stats.shapiro(calls_BC)
print(f"Group A: p-value = {shapiro_A.pvalue:.4f} -> {'normally distributed' if shapiro_A.pvalue >= alpha else 'not normally distributed'}")
print(f"Group B+C: p-value = {shapiro_BC.pvalue:.4f} -> {'normally distributed' if shapiro_BC.pvalue >= alpha else 'not normally distributed'}")

# Levene test for equal variances
print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_p = stats.levene(calls_A, calls_BC).pvalue
print(f"Levene p-value = {levene_p:.4f} -> {'variances are similar' if levene_p >= alpha else 'variances differ'}")

print(f"\n=== TEST RESULTS ===")
# Choose statistical test
normal_A = shapiro_A.pvalue >= alpha
normal_BC = shapiro_BC.pvalue >= alpha
equal_var = levene_p >= alpha

if normal_A and normal_BC and equal_var:
    print("Using independent t-test (parametric)")
    stat, p_value = stats.ttest_ind(calls_A, calls_BC, equal_var=True)
else:
    print("Using Mann-Whitney U test (non-parametric)")
    stat, p_value = stats.mannwhitneyu(calls_A, calls_BC, alternative='two-sided')

print(f"Test statistic = {stat:.4f}")
print(f"P-value = {p_value:.4f}")

if p_value < alpha:
    print(f"SIGNIFICANT difference: p = {p_value:.4f} < {alpha}")
    print("There are statistically significant differences in teacher solicitation between Group A and Group B+C")
else:
    print(f"No significant difference: p = {p_value:.4f} >= {alpha}")


### 5.3/ Test A VS B VS C

Is there a difference in teacher solicitation between students from groups A, B, and C?

In [ ]:
# Descriptive statistics
average_calls_A = calls_A.mean()
average_calls_B = calls_B.mean()
average_calls_C = calls_C.mean()

diff_B_A = average_calls_A - average_calls_B
diff_C_A = average_calls_A - average_calls_C

percent_reduction_B = (diff_B_A / average_calls_A) * 100
percent_reduction_C = (diff_C_A / average_calls_A) * 100

print("\n=== DESCRIPTIVE STATISTICS ===")
print(f"Group A (n={len(calls_A)}): M = {average_calls_A:.2f}, SD = {calls_A.std():.2f}")
print(f"Group B (n={len(calls_B)}): M = {average_calls_B:.2f}, SD = {calls_B.std():.2f}")
print(f".   B vs A: Difference = {diff_B_A:.2f}, Percentage reduction = {percent_reduction_B:.2f}%")
print(f"Group C (n={len(calls_C)}): M = {average_calls_C:.2f}, SD = {calls_C.std():.2f}")
print(f".   C vs A: Difference = {diff_C_A:.2f}, Percentage reduction = {percent_reduction_C:.2f}%")

alpha = 0.05
# Shapiro-Wilk normality test
print(f"\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_A = stats.shapiro(calls_A)
shapiro_B = stats.shapiro(calls_B)
shapiro_C = stats.shapiro(calls_C)
print(f"Group A: p-value = {shapiro_A.pvalue:.4f}")
print(f"Group B: p-value = {shapiro_B.pvalue:.4f}")
print(f"Group C: p-value = {shapiro_C.pvalue:.4f}")

# Check if all groups are normally distributed
all_normal = all(p >= alpha for p in [shapiro_A.pvalue, shapiro_B.pvalue, shapiro_C.pvalue])

if all_normal:
    print("All groups follow normal distribution -> ANOVA is applicable.")
else:
    print("At least one group is not normally distributed -> Prefer Kruskal-Wallis test.")

# Test for homogeneity of variances (Levene's test)
print(f"\n=== HOMOGENEITY OF VARIANCES (Levene) ===")

levene_test = stats.levene(calls_A, calls_B, calls_C)
print(f"Levene's test for homogeneity of variances: p-value = {levene_test.pvalue:.4f}")

if levene_test.pvalue >= alpha:
    print("Variances are homogeneous")
else:
    print("Variances are not homogeneous")

# Prepare data for overall tests
all_calls = pd.concat([
    pd.DataFrame({'group': 'A', 'calls': calls_A}),
    pd.DataFrame({'group': 'B', 'calls': calls_B}),
    pd.DataFrame({'group': 'C', 'calls': calls_C})
])

# ANOVA (parametric)
anova_result = stats.f_oneway(calls_A, calls_B, calls_C)
print(f"\n=== ONE-WAY ANOVA RESULTS ===")
print(f"F-statistic: {anova_result.statistic:.4f}")
print(f"P-value: {anova_result.pvalue:.4f}")

# Kruskal-Wallis test (non-parametric)
kruskal_result = stats.kruskal(calls_A, calls_B, calls_C)
print(f"\n=== KRUSKAL-WALLIS TEST RESULTS ===")
print(f"H-statistic: {kruskal_result.statistic:.4f}")
print(f"P-value: {kruskal_result.pvalue:.4f}")

# Interpretation of overall test
if all_normal and levene_test.pvalue >= 0.05:
    overall_pvalue = anova_result.pvalue
    test_used = "ANOVA"
else:
    overall_pvalue = kruskal_result.pvalue
    test_used = "Kruskal-Wallis"

print(f"\n=== OVERALL INTERPRETATION (using {test_used}) ===")
if overall_pvalue < alpha:
    print(f"SIGNIFICANT: p-value = {overall_pvalue:.4f} < alpha = {alpha}")
    print("   There are statistically significant differences in teacher solicitation between at least two groups")
else:
    print(f"NOT SIGNIFICANT: p-value = {overall_pvalue:.4f} > alpha = {alpha}")
    print("   No statistically significant differences in teacher solicitation between groups")

# POST-HOC TESTS (if significant overall difference)
if overall_pvalue < alpha:
    print(f"\n=== POST-HOC PAIRWISE COMPARISONS ===")
    
    # For parametric data: Tukey HSD
    if test_used == "ANOVA":
        print("Tukey HSD Post-hoc Test:")
        tukey = pairwise_tukeyhsd(endog=all_calls['calls'], groups=all_calls['group'], alpha=alpha)
        print(tukey)
        
    # For non-parametric data: Mann-Whitney U tests with Bonferroni correction
    else:
        print("Mann-Whitney U tests with Bonferroni correction:")
        pairs = [('A', 'B'), ('A', 'C'), ('B', 'C')]
        bonferroni_alpha = alpha / len(pairs)
        
        for pair in pairs:
            group1, group2 = pair
            data1 = calls_A if group1 == 'A' else (calls_B if group1 == 'B' else calls_C)
            data2 = calls_A if group2 == 'A' else (calls_B if group2 == 'B' else calls_C)
            
            u_stat, p_value = stats.mannwhitneyu(data1, data2)
            adjusted_p = p_value * len(pairs)
            significance = "SIGNIFICANT" if adjusted_p < alpha else "not significant"
            
            print(f"  {group1} vs {group2}: p = {p_value:.4f}, adjusted p = {adjusted_p:.4f} -> {significance}")

## 6/ Activity following feedback (Q1.4) [B/C]

### 6.1/ Calculation of student activity following feedback

1/ Following a request to the digital assistant, we categorize the subsequent sequence into four categories:
- `progression`: the student managed to progress in the game (first increment of game_progression or reach of next level) without calling the teacher or the digital assistant again
- `teacher_call`: the student requested help from the teacher (without having progressed in the game)
- `assistant_call`: the student requested help from the digital assistant again (without having progressed in the game)
- `other`: other cases, for example, the experiment ends

Note that we do not have in the traces the start date of teacher interventions but only the end date. This is why, when we detect a progression in a sequence, we search in the following 120 seconds for a trace of the end of a teacher intervention. If we find one, we reclassify the sequence as `teacher_call`.

2/ Moreover, when a progression sequence occurs, we compute:
- the progression gain (difference in game_progression)
- the sequence of actions before the progression occurred (see `categorize_action()` function bellow)

In [ ]:
# Filter for groups B and C
groups_BC_interaction = interaction_data[
    interaction_data[int_const.GROUP_ID_DATA_KEY].isin([int_const.GROUP_B, int_const.GROUP_C])
]

# Sort by student and date to analyze sequences
groups_BC_interaction = groups_BC_interaction.sort_values([
    int_const.GAME_ID_DATA_KEY, 
    int_const.DATE_DATA_KEY
]).reset_index(drop=True)

# Filter all feedback reception events
feedback_events = groups_BC_interaction[
    (groups_BC_interaction[int_const.ACTION_DATA_KEY] == int_const.RECEIVED_ACTION) &
    (groups_BC_interaction[int_const.OBJECT_DATA_KEY] == int_const.ASSISTANT_HELP_OBJECT)
].copy()


# Define action categories
def categorize_action(row):
    action = row[int_const.ACTION_DATA_KEY]
    object_name = row[int_const.OBJECT_DATA_KEY]
    # Display of the memo
    if action == int_const.DISPLAYED_ACTION:
        return "memo_displayed"
    # Copy of feedback content
    elif (action == int_const.COPIED_ACTION and 
          object_name == int_const.ASSISTANT_MESSAGES_SECTION_OBJECT):
        return "feedback_content_copied"
    # Copy of the memo content
    elif (action == int_const.COPIED_ACTION and 
          object_name in [
              int_const.BASE_PROGRAM_SECTION_OBJECT,
              int_const.BASE_ERROR_SECTION_OBJECT,
              int_const.BASE_STRUCTURE_SECTION_OBJECT,
              int_const.BASE_COMMENT_SECTION_OBJECT,
              int_const.VAR_CREATION_SECTION_OBJECT,
              int_const.VAR_MODIFICATION_SECTION_OBJECT,
              int_const.VAR_USAGE_SECTION_OBJECT,
              int_const.VAR_TYPE_SECTION_OBJECT,
              int_const.CONDI_1BRAN_SECTION_OBJECT,
              int_const.CONDI_2BRAN_SECTION_OBJECT,
              int_const.CONDI_3BRAN_SECTION_OBJECT,
              int_const.FOR_SIMPLE_SECTION_OBJECT,
              int_const.FOR_COUNTER_0_SECTION_OBJECT,
              int_const.FOR_COUNTER_N_SECTION_OBJECT,
              int_const.WHILE_SIMPLE_SECTION_OBJECT
          ]):
        return "memo_content_copied"
    # Copy of code editor
    elif (action == int_const.COPIED_ACTION and 
          object_name == int_const.CODE_EDITOR_SECTION_OBJECT):
        return "code_editor_copied"
    # Copy of control function
    elif (action == int_const.COPIED_ACTION and 
          object_name == int_const.CONTROL_FUNCTION_SECTION_OBJECT):
        return "control_function_copied"
    # Code paste
    elif action == int_const.PASTED_ACTION:
        return "code_pasted"
    # Launch of a correct program
    elif (action == int_const.LAUNCHED_ACTION and 
          object_name in [
              int_const.FULLY_EXECUTED_PROGRAM_OBJECT,
              int_const.LEVEL_COMPLETED_PROGRAM_OBJECT
          ]):
        return "correct_program_launched"
    # Launch of an erroneous program
    elif (action == int_const.LAUNCHED_ACTION and 
          object_name in [
              int_const.LEVEL_LOST_PROGRAM_OBJECT,
              int_const.TOO_MANY_LINES_ERROR_PROGRAM_OBJECT,
              int_const.GAME_ERROR_PROGRAM_OBJECT,
              int_const.SYNTACTIC_ERROR_PROGRAM_OBJECT,
              int_const.SEMANTIC_ERROR_PROGRAM_OBJECT
          ]):
        return "erroneous_program_launched"
    # Launch of a stopped program
    elif (action == int_const.LAUNCHED_ACTION and 
          object_name == int_const.USER_STOPPED_PROGRAM_OBJECT):
        return "stopped_program_launched"
    # Leave to visit a completed level
    elif action == int_const.LEAVED_ACTION:
        return "level_leaved"
    # Back from a completed level
    elif action in [int_const.RESUMED_ACTION, int_const.RESTARTED_ACTION]:
        return "level_resumed"
    # End of the level (not a real action)
    elif action == int_const.ENDED_ACTION:
        return "level_ended"
    # Assistant help called
    elif (action == int_const.ASKED_ACTION and 
          object_name == int_const.ASSISTANT_HELP_OBJECT):
        return "assistant_help_called"
    # Assistant help received
    elif (action == int_const.RECEIVED_ACTION and 
          object_name == int_const.ASSISTANT_HELP_OBJECT):
        return "assistant_help_received"
    # Teacher help received
    elif (action == int_const.RECEIVED_ACTION and 
          object_name == int_const.TEACHER_HELP_OBJECT):
        return "teacher_help_received"
    # Change of speed cursor
    elif (action == int_const.CHANGED_ACTION and 
          object_name == int_const.EXECUTION_SPEED_SLIDE_OBJECT):
        return "speed_cursor_changed"
    else:
        return "other"


feedback_analysis = []
# Iterate on each feedback event
for _, feedback in feedback_events.iterrows():
    game_id = feedback[int_const.GAME_ID_DATA_KEY]
    group_id = feedback[int_const.GROUP_ID_DATA_KEY]
    feedback_date = feedback[int_const.DATE_DATA_KEY]
    feedback_level = feedback[int_const.LEVEL_DATA_KEY]
    feedback_progression = feedback[int_const.GAME_PROGRESSION_DATA_KEY]
    
    # Get student's traces after this feedback
    student_traces = groups_BC_interaction[
        (groups_BC_interaction[int_const.GAME_ID_DATA_KEY] == game_id) &
        (groups_BC_interaction[int_const.DATE_DATA_KEY] > feedback_date)
    ]
    
    actions_after_feedback = []
    sequence_outcome = "other"  # Default outcome
    progression_gain = 0  # Initialize progression gain
    progression_date = None  # Track when progression happened
    
    # Iterate on post-feedback traces
    for _, trace in student_traces.iterrows():
        current_progression = trace[int_const.GAME_PROGRESSION_DATA_KEY]
        current_level = trace[int_const.LEVEL_DATA_KEY]
        
        # Check for teacher call progression
        if (trace[int_const.ACTION_DATA_KEY] == int_const.RECEIVED_ACTION and 
            trace[int_const.OBJECT_DATA_KEY] == int_const.TEACHER_HELP_OBJECT):
            sequence_outcome = "teacher_call"
            break
        
        # Check for new assistant call progression
        if (trace[int_const.ACTION_DATA_KEY] == int_const.ASKED_ACTION and 
            trace[int_const.OBJECT_DATA_KEY] == int_const.ASSISTANT_HELP_OBJECT):
            sequence_outcome = "assistant_call"
            break
        
        # Check if progression increased (in same level) or level increased
        progression_increase = (current_level == feedback_level and 
                               current_progression > feedback_progression)
        level_increase = current_level > feedback_level

        # Check if progression achieved
        if progression_increase or level_increase:
            sequence_outcome = "progression"
            # Calculate progression gain
            if level_increase:
                progression_gain = 100 - feedback_progression
            else:
                progression_gain = current_progression - feedback_progression
            
            # Store the date when progression happened
            progression_date = trace[int_const.DATE_DATA_KEY]
            
            # We break without adding the action that make progression detection
            # That's because in the Pyrates trace system, the progression is reported in the next trace
            # i.e. each trace report the progression before its action
            break
        
        # Categorize and count all sequence actions
        action_type = categorize_action(trace)
        # Ignore level_end which is not really an action
        if action_type not in ["level_ended"]:
            actions_after_feedback.append(action_type)
    
    # Check for teacher call within 120 seconds after detected progression
    if sequence_outcome == "progression" and progression_date is not None:
        # Look for teacher help in the 120 seconds following progression
        teacher_help_after_progression = groups_BC_interaction[
            (groups_BC_interaction[int_const.GAME_ID_DATA_KEY] == game_id) &
            (groups_BC_interaction[int_const.DATE_DATA_KEY] > progression_date) &
            (groups_BC_interaction[int_const.DATE_DATA_KEY] <= progression_date + pd.Timedelta(seconds=120)) &
            (groups_BC_interaction[int_const.ACTION_DATA_KEY] == int_const.RECEIVED_ACTION) &
            (groups_BC_interaction[int_const.OBJECT_DATA_KEY] == int_const.TEACHER_HELP_OBJECT)
        ]
        
        if not teacher_help_after_progression.empty:
            sequence_outcome = "teacher_call"
    
    # Store analysis for this feedback event
    feedback_analysis.append({
        'game_id': game_id,
        'group': group_id,
        'feedback_level': feedback_level,
        'feedback_progression': feedback_progression,
        'actions_count': len(actions_after_feedback),
        'actions_sequence': actions_after_feedback,
        'sequence_outcome': sequence_outcome,
        'progression_gain': progression_gain,
    })

# Convert to DataFrame
analysis_df = pd.DataFrame(feedback_analysis)
# Export
analysis_df.to_excel(f"../debug/debug_activity_following_feedback.xlsx")

print(f"Total sequences analyzed: {len(feedback_analysis)}")

### 6.2/ Feedback outcomes

What is the outcome of the action sequences following the reception of feedback depending on the groups?

In [ ]:
# Analysis for each group
for group in [int_const.GROUP_B, int_const.GROUP_C]:
    group_data = analysis_df[analysis_df['group'] == group]
    
    print(f"\n=======================================")
    print(f"GROUP {group} FEEDBACK OUTCOMES")
    print(f"========================================")
    print(f"Total feedback sequences: {len(group_data)}")
    
    outcome_distribution = group_data['sequence_outcome'].value_counts()
    for outcome, count in outcome_distribution.items():
        percentage = (count / len(group_data)) * 100
        print(f"   {outcome}: {count} sequences ({percentage:.1f}%)")

In [ ]:
groups = [int_const.GROUP_B, int_const.GROUP_C]

# Calculate outcome frequencies across both groups to determine order
overall_counts = analysis_df['sequence_outcome'].value_counts()
sorted_outcomes = overall_counts.index.tolist()  # Outcomes sorted by frequency

# Prepare a DataFrame to store percentages for each group
percentage_df = pd.DataFrame(index=sorted_outcomes, columns=groups, dtype=float)

for group in groups:
    # Filter data for the current group
    group_data = analysis_df[analysis_df['group'] == group]
    total = len(group_data)
    counts = group_data['sequence_outcome'].value_counts()
    
    for outcome in sorted_outcomes:
        # Calculate percentage of each outcome for the group
        percentage_df.loc[outcome, group] = (counts.get(outcome, 0) / total) * 100

# Parameters for side-by-side bar chart
x = np.arange(len(sorted_outcomes))
width = 0.4  # width of the bars

# Create the figure and axes
fig, ax = plt.subplots(figsize=(8, 5))

# Plot bars for group B and C
bars_B = ax.bar(x - width/2, percentage_df[int_const.GROUP_B], width, label='Group B', color='skyblue')
bars_C = ax.bar(x + width/2, percentage_df[int_const.GROUP_C], width, label='Group C', color='salmon')

# Set x-axis labels and rotation
ax.set_xticks(x)
ax.set_xticklabels(sorted_outcomes, rotation=30, ha='right')

# Add axis labels and title
ax.set_xlabel("Feedback outcome", fontsize=12)
ax.set_ylabel("Percentage (%)", fontsize=12)
ax.set_ylim(0, 60)
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add percentage labels on top of each bar
for bars in [bars_B, bars_C]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # offset label slightly above bar
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)

# Add legend
ax.legend()

# Adjust layout, save figure and display
plt.tight_layout()
plt.savefig("../outputs/feedback_outcomes_BC.png", dpi=300, bbox_inches="tight")
plt.show()


### 6.3/ Progression gain

What is the progression gain resulting of a progression outcome depending on the groups ?

In [ ]:
# Extract progression-only data
progress_B = analysis_df[(analysis_df['group'] == int_const.GROUP_B) & 
                         (analysis_df['sequence_outcome'] == 'progression')]

progress_C = analysis_df[(analysis_df['group'] == int_const.GROUP_C) & 
                         (analysis_df['sequence_outcome'] == 'progression')]

data_B = progress_B["progression_gain"].dropna()
data_C = progress_C["progression_gain"].dropna()

# Descriptive statistics
mean_B = data_B.mean()
mean_C = data_C.mean()
std_B = data_B.std()
std_C = data_C.std()

# Difference in means
mean_difference = mean_C - mean_B
# Percentage increase (relative to group B)
percent_increase = (mean_difference / mean_B) * 100

print("=== DESCRIPTIVE STATISTICS ===")
print(f"Group B (n={len(data_B)}): M = {mean_B:.2f}, SD = {std_B:.2f}")
print(f"Group C (n={len(data_C)}): M = {mean_C:.2f}, SD = {std_C:.2f}")
print(f".   Difference (C - B): {mean_difference:.2f}")
print(f".   Percentage increase from B to C: {percent_increase:.2f}%")

alpha = 0.05
# Normality test
print("\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_B = stats.shapiro(data_B)
shapiro_C = stats.shapiro(data_C)
print(f"Group B: p-value = {shapiro_B.pvalue:.4f} -> "
        f"{'normally distributed' if shapiro_B.pvalue >= alpha else 'not normally distributed'}")
print(f"Group C: p-value = {shapiro_C.pvalue:.4f} -> "
        f"{'normally distributed' if shapiro_C.pvalue >= alpha else 'not normally distributed'}")

# Levene test
print("\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_p = stats.levene(data_B, data_C).pvalue

print(f"Levene p-value = {levene_p:.4f} -> "
        f"{'variances are similar' if levene_p >= alpha else 'variances differ'}")

print(f"\n=== TEST RESULTS ===")
# Choose appropriate test
normal_B = shapiro_B.pvalue >= alpha
normal_C = shapiro_C.pvalue >= alpha
equal_var = levene_p >= alpha

if normal_B and normal_C and equal_var:
    print("Using independent t-test (parametric)")
    stat, p_value = stats.ttest_ind(data_B, data_C, equal_var=True)
else:
    print("Using Mann-Whitney U test (non-parametric)")
    stat, p_value = stats.mannwhitneyu(data_B, data_C, alternative='two-sided')

print(f"Test statistic = {stat:.4f}")
print(f"P-value = {p_value:.4f}")

if p_value < alpha:
    print(f"SIGNIFICANT difference: p = {p_value:.4f} < {alpha}")
else:
    print(f"No significant difference: p = {p_value:.4f} >= {alpha}")

### 6.4/ Number of actions before progression

How many actions do students need to achieve a progression in the game following feedback depending on the groups?

In [ ]:
# Extract progression-only data
progress_B = analysis_df[(analysis_df['group'] == int_const.GROUP_B) & 
                         (analysis_df['sequence_outcome'] == 'progression')]

progress_C = analysis_df[(analysis_df['group'] == int_const.GROUP_C) & 
                         (analysis_df['sequence_outcome'] == 'progression')]

data_B = progress_B["actions_count"].dropna()
data_C = progress_C["actions_count"].dropna()

# Descriptive statistics
mean_B = data_B.mean()
mean_C = data_C.mean()
std_B = data_B.std()
std_C = data_C.std()

# Difference in means
mean_difference = mean_C - mean_B
# Percentage increase (relative to group B)
percent_increase = (mean_difference / mean_B) * 100

print("=== DESCRIPTIVE STATISTICS ===")
print(f"Group B (n={len(data_B)}): M = {mean_B:.2f}, SD = {std_B:.2f}")
print(f"Group C (n={len(data_C)}): M = {mean_C:.2f}, SD = {std_C:.2f}")
print(f".   Difference (C - B): {mean_difference:.2f}")
print(f".   Percentage increase from B to C: {percent_increase:.2f}%")

alpha = 0.05
# Normality test
print("\n=== NORMALITY TESTS (Shapiro-Wilk) ===")
shapiro_B = stats.shapiro(data_B)
shapiro_C = stats.shapiro(data_C)
print(f"Group B: p-value = {shapiro_B.pvalue:.4f} -> "
        f"{'normally distributed' if shapiro_B.pvalue >= 0.05 else 'not normally distributed'}")
print(f"Group C: p-value = {shapiro_C.pvalue:.4f} -> "
        f"{'normally distributed' if shapiro_C.pvalue >= 0.05 else 'not normally distributed'}")

# Levene test
print("\n=== HOMOGENEITY OF VARIANCES (Levene) ===")
levene_p = stats.levene(data_B, data_C).pvalue

print(f"Levene p-value = {levene_p:.4f} -> "
        f"{'variances are similar' if levene_p >= 0.05 else 'variances differ'}")

print(f"\n=== TEST RESULTS ===")
# Choose appropriate test
normal_B = shapiro_B.pvalue >= alpha
normal_C = shapiro_C.pvalue >= alpha
equal_var = levene_p >= alpha

if normal_B and normal_C and equal_var:
    print("Using independent t-test (parametric)")
    stat, p_value = stats.ttest_ind(data_B, data_C, equal_var=True)
else:
    print("Using Mann-Whitney U test (non-parametric)")
    stat, p_value = stats.mannwhitneyu(data_B, data_C, alternative='two-sided')

print(f"Test statistic = {stat:.4f}")
print(f"P-value = {p_value:.4f}")

if p_value < alpha:
    print(f"SIGNIFICANT difference: p = {p_value:.4f} < {alpha}")
else:
    print(f"No significant difference: p = {p_value:.4f} >= {alpha}")


### 6.5/ Action distribution in progression sequences

What type of actions do students perform before achieving a progression in the game following feedback depending on the groups?

In [ ]:
# Analysis for each group
for group in [int_const.GROUP_B, int_const.GROUP_C]:
    group_data = analysis_df[analysis_df['group'] == group]
    print(f"\n=============================")
    print(f"GROUP {group} ACTION DISTRIBUTION")
    print(f"=============================")
    
    # Analysis for progression outcomes
    progression_data = group_data[group_data['sequence_outcome'] == 'progression']
    
    if len(progression_data) > 0:
        all_actions = []
        for action_list in progression_data['actions_sequence']:
            all_actions.extend(action_list)
        action_counts = pd.Series(all_actions).value_counts()
        total_actions = len(all_actions)
            
        for action, count in action_counts.head(10).items():  # Top 10 actions
            percentage = (count / total_actions) * 100
            print(f"   {action}: {count} times ({percentage:.1f}%)")
        
        print(f"   Total actions analyzed: {total_actions}")    

In [ ]:
# Groups to compare
groups = [int_const.GROUP_B, int_const.GROUP_C]

# Collect all actions across both groups during progression to determine overall frequency
all_actions_overall = []
for group in groups:
    group_data = analysis_df[(analysis_df['group'] == group) & (analysis_df['sequence_outcome'] == 'progression')]
    for action_list in group_data['actions_sequence']:
        all_actions_overall.extend(action_list)

# Get top 6 actions sorted by overall frequency
overall_counts = pd.Series(all_actions_overall).value_counts()
sorted_actions = overall_counts.head(6).index.tolist()  # Keep only top 6 actions

# Prepare a DataFrame to store percentages for each group
percentage_df = pd.DataFrame(index=sorted_actions, columns=groups, dtype=float)

for group in groups:
    # Filter progression sequences for the current group
    group_data = analysis_df[(analysis_df['group'] == group) & (analysis_df['sequence_outcome'] == 'progression')]
    
    # Flatten all actions
    all_actions = []
    for action_list in group_data['actions_sequence']:
        all_actions.extend(action_list)
    
    total_actions = len(all_actions)
    action_counts = pd.Series(all_actions).value_counts()
    
    # Calculate percentages for each action
    for action in sorted_actions:
        percentage_df.loc[action, group] = (action_counts.get(action, 0) / total_actions) * 100 if total_actions > 0 else 0

# Parameters for side-by-side bar chart
x = np.arange(len(sorted_actions))
width = 0.40  # width of the bars

# Create figure and axes
fig, ax = plt.subplots(figsize=(8, 5))

# Plot bars for group B and C
bars_B = ax.bar(x - width/2, percentage_df['B'], width, label='Group B', color='skyblue')
bars_C = ax.bar(x + width/2, percentage_df['C'], width, label='Group C', color='salmon')

# Set x-axis labels and rotation
ax.set_xticks(x)
ax.set_xticklabels(sorted_actions, rotation=30, ha='right')

# Add axis labels and title
ax.set_xlabel("Action type", fontsize=12)
ax.set_ylabel("Percentage (%)", fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.set_ylim(0, 37)

# Add percentage labels on top of each bar
for bars in [bars_B, bars_C]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # offset label slightly above bar
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)

# Add legend
ax.legend()

# Adjust layout, save figure, and display
plt.tight_layout()
plt.savefig("../outputs/action_types_BC.png", dpi=300, bbox_inches="tight")
plt.show()


### 6.6/ Action patterns in progression sequences 

What are the most frequent action patterns in progression sequences depending on groups?

In [ ]:
top_n_display = 10
# Filter only progression events for groups B and C
progress_B = analysis_df[
    (analysis_df['group'] == int_const.GROUP_B) &
    (analysis_df['sequence_outcome'] == 'progression')
]

progress_C = analysis_df[
    (analysis_df['group'] == int_const.GROUP_C) &
    (analysis_df['sequence_outcome'] == 'progression')
]

# Extract full action sequences as tuples (so they are hashable)
sequences_B = progress_B['actions_sequence'].apply(tuple)
sequences_C = progress_C['actions_sequence'].apply(tuple)

# Count most common sequences for each group (top n)
top_sequences_B = sequences_B.value_counts().head(top_n_display)
top_sequences_C = sequences_C.value_counts().head(top_n_display)

# Compute total for percentages
total_B = len(sequences_B)
total_C = len(sequences_C)

# Display results
print(f"\n==================================")
print(f"Top {top_n_display} Action Sequences – Group B")
print(f"==================================")
for seq, count in top_sequences_B.items():
    pct = (count / total_B) * 100
    print(f"- {list(seq)} → {count} occurrences ({pct:.1f}%)")

print(f"\n==================================")
print(f"Top {top_n_display} Action Sequences – Group C")
print(f"==================================")
for seq, count in top_sequences_C.items():
    pct = (count / total_C) * 100
    print(f"- {list(seq)} → {count} occurrences ({pct:.1f}%)")


In [ ]:
top_n_plot = 6

# Count most common sequences for each group (TOP6 for plotting)
top_sequences_B_plot = sequences_B.value_counts().head(top_n_plot)
top_sequences_C_plot = sequences_C.value_counts().head(top_n_plot)

# Labels and percentages for plotting
seq_labels_B = ['\n'.join(seq) for seq in top_sequences_B_plot.index]
seq_labels_C = ['\n'.join(seq) for seq in top_sequences_C_plot.index]

percent_B = [(count / total_B) * 100 for count in top_sequences_B_plot.values]
percent_C = [(count / total_C) * 100 for count in top_sequences_C_plot.values]

# X positions
x_B = np.arange(len(top_sequences_B_plot))
x_C = np.arange(len(top_sequences_C_plot))
width = 0.4

fig, ax = plt.subplots(figsize=(8, 5))

# Plot bars
bars_B = ax.bar(x_B - width/2, percent_B, width, label='Group B', color='skyblue')
bars_C = ax.bar(x_C + width/2, percent_C, width, label='Group C', color='salmon')

# X-axis labels
ax.set_xticks(np.arange(top_n_plot))
labels_combined = seq_labels_B + seq_labels_C
ax.set_xticklabels(labels_combined[:top_n_plot], rotation=30, ha='right')

# Axis labels and title
ax.set_xlabel("Top Action Sequences", fontsize=12)
ax.set_ylabel("Percentage (%)", fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)
ax.set_ylim(0, max(max(percent_B), max(percent_C))+2)

# Add percentage labels
for bars in [bars_B, bars_C]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

# Legend
ax.legend()
plt.tight_layout()
plt.savefig("../outputs/top_action_sequences_BC.png", dpi=300, bbox_inches="tight")
plt.show()